<a href="https://colab.research.google.com/github/luissanty2/TFGRecursos/blob/main/Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creación del dataset
Este cuaderno de colab está destinado a la creación del set de datos inicial a partir de archivos csv oficiales obtenidos de [datahub](https://datahub.io/sports-data/atp-world-tour-tennis-data). 

Todos los archivos han sido subidos previamente a un repositorio de github previamente para trabajar fácilmente con los ficheros sin tener la necesidad de subirlos a colab para poder modificar cualquier cosa necesaria del dataset.

Ningún csv, salvo el utilizado para el ranking, que ha sido modificado para que pesara menos, guardado en github ha sido alterado.

In [51]:
import pandas as pd
import numpy as np

En primer lugar se modifican algunas columnas para relativizar ciertos datos y hacerlos cómodos de utilizar, y se unen los datos de puntuaciones y estadísticas de partidos, conservando solamente columnas útiles.

In [52]:
df_scores1 = pd.read_csv("https://media.githubusercontent.com/media/luissanty2/TFGRecursos/main/datasets/match_scores_1991-2016_unindexed_csv.csv", encoding = 'latin1')
df_scores2 = pd.read_csv("https://media.githubusercontent.com/media/luissanty2/TFGRecursos/main/datasets/match_scores_2017_unindexed_csv.csv", encoding = 'latin1')

df_scores = pd.concat([df_scores1, df_scores2], axis=0)


df_stats1 = pd.read_csv("https://media.githubusercontent.com/media/luissanty2/TFGRecursos/main/datasets/match_stats_1991-2016_unindexed_csv.csv", encoding = 'latin1')
df_stats2 = pd.read_csv("https://media.githubusercontent.com/media/luissanty2/TFGRecursos/main/datasets/match_stats_2017_unindexed_csv.csv", encoding = 'latin1')

df_stats = pd.concat([df_stats1, df_stats2], axis=0)


df_stats['winner_double_faults'] = df_stats['winner_double_faults'] / df_stats['winner_service_points_total']
df_stats['loser_double_faults'] = df_stats['loser_double_faults'] / df_stats['loser_service_points_total']

df_stats['winner_first_serves_in'] = df_stats['winner_first_serves_in'] / df_stats['winner_first_serves_total']
df_stats['loser_first_serves_in'] = df_stats['loser_first_serves_in'] / df_stats['loser_first_serves_total']

df_stats['winner_first_serve_points_won'] = df_stats['winner_first_serve_points_won'] / df_stats['winner_first_serve_points_total']
df_stats['loser_first_serve_points_won'] = df_stats['loser_first_serve_points_won'] / df_stats['loser_first_serve_points_total']

df_stats['winner_second_serves_in'] = 1 - (df_stats['winner_first_serves_in'] + df_stats['winner_double_faults'])
df_stats['loser_second_serves_in'] = 1 - (df_stats['loser_first_serves_in'] + df_stats['loser_double_faults'])


df_matchs = pd.merge(df_stats[['match_id','match_duration','winner_aces', 'loser_aces','winner_double_faults','loser_double_faults','winner_first_serves_in','loser_first_serves_in', 'loser_first_serve_points_won', 'winner_first_serve_points_won',
                               'winner_second_serves_in', 'loser_second_serves_in']],
                     df_scores[['tourney_year_id','match_id','tourney_round_name','winner_name','winner_player_id','loser_name','loser_player_id','winner_sets_won','loser_sets_won','winner_games_won','loser_games_won']],
                     on='match_id')


df_tournaments = pd.read_csv("https://media.githubusercontent.com/media/luissanty2/TFGRecursos/main/datasets/tournaments_1877-2017_unindexed_csv.csv", encoding = 'latin1')


df = pd.merge(df_matchs, df_tournaments[['tourney_surface','tourney_year_id','tourney_dates','tourney_year', 'tourney_month']], on=['tourney_year_id'])

Ahora se añade el elo-score (puntos en la clasificación ATP) al set de datos anterior.

In [53]:
df_ranking = pd.read_csv('https://media.githubusercontent.com/media/luissanty2/TFGRecursos/main/datasets/ranking_simplificado.csv')


df = pd.merge(df, df_ranking[['ranking_points', 'week_year', 'week_month','player_id']], how='left', left_on=['tourney_year', 'tourney_month', 'winner_player_id'], right_on=['week_year','week_month','player_id'])
df['winner_ranking_points'] = df['ranking_points']


df = df.drop(columns=['ranking_points', 'week_year', 'week_month','player_id'])


df = pd.merge(df, df_ranking[['ranking_points', 'week_year', 'week_month','player_id']], how='left', left_on=['tourney_year', 'tourney_month', 'loser_player_id'], right_on=['week_year','week_month','player_id'])
df['loser_ranking_points'] = df['ranking_points']

df = df.drop(columns=['ranking_points', 'week_year', 'week_month','player_id'])

df = df.drop_duplicates(subset='match_id', keep="first")

<ipython-input-53-cd29aa5c1534>:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ranking = pd.read_csv('https://media.githubusercontent.com/media/luissanty2/TFGRecursos/main/datasets/ranking_simplificado.csv')


Se eliminan los partidos jugados en moqueta (carpet), ya que, al menos actualmente, no se juegan competiciones oficiales en moqueta.

Se eliminan columnas inútiles que han sido utilizadas para fusionar diferentes tablas.

Se eliminan filas con nulos.

Se renombra una columna para mayor comodidad en su uso.

Se resetean los índices.

In [54]:
df = df.drop(df[df.tourney_surface == "Carpet"].index)
df = df.drop(columns=['tourney_year','match_id','tourney_year_id', 'winner_player_id', 'loser_player_id','tourney_month', 'tourney_dates'])
df = df.dropna()
df.rename(columns={"tourney_surface": "surface"}, inplace=True)
df = df.reset_index(drop=True)

Debido a como se realizará la regresión, es necesario tener una columna objetivo (y), así que se renombran las columnas (winner, loser => p1, p2) y se cambian algunos datos de columna para no solo tener y=0 (y==0 gana p1, y==1 gana p2):

Se cambia:

| winner_name | loser_name | y |
|----------|----------|----------|
| Rafael Nadal | Roger Federer   | 0   |
| Novak Djokovic | Andy Murray   | 0   |

a

| p1_name | p2_name | y |
|----------|----------|----------|
| Rafael Nadal | Roger Federer   | 0   |
| Andy Murray | Novak Djokovic   | 1   |

In [55]:
df.columns = df.columns.str.replace('winner', 'p1')
df.columns = df.columns.str.replace('loser', 'p2')

idx = (df.index < (df.shape[0]/2))

df.loc[idx,['p1_aces','p2_aces']] = df.loc[idx,['p2_aces','p1_aces']].values
df.loc[idx,['p1_double_faults','p2_double_faults']] = df.loc[idx,['p2_double_faults','p1_double_faults']].values
df.loc[idx,['p1_first_serves_in' , 'p2_first_serves_in']] = df.loc[idx,['p2_first_serves_in' ,'p1_first_serves_in' ]].values
df.loc[idx,['p1_first_serve_points_won' , 'p2_first_serve_points_won']] = df.loc[idx,['p2_first_serve_points_won' ,'p1_first_serve_points_won' ]].values
df.loc[idx,['p1_second_serves_in' , 'p2_second_serves_in']] = df.loc[idx,['p2_second_serves_in' ,'p1_second_serves_in' ]].values
df.loc[idx,['p1_sets_won', 'p2_sets_won']] = df.loc[idx,['p2_sets_won' , 'p1_sets_won']].values  
df.loc[idx,[ 'p1_games_won','p2_games_won' ]] = df.loc[idx,['p2_games_won' ,'p1_games_won' ]].values
df.loc[idx,['p1_ranking_points' ,'p2_ranking_points' ]] = df.loc[idx,['p2_ranking_points' ,'p1_ranking_points' ]].values
df.loc[idx,['p1_name','p2_name']] = df.loc[idx,['p2_name','p1_name']].values

df['y'] = 0
df.loc[idx, 'y'] = 1

Finalmente, se guarda el dataset.

In [56]:
df.to_csv('dataset_final.csv')
df.head()

,match_duration,p1_aces,p2_aces,p1_double_faults,p2_double_faults,p1_first_serves_in,p2_first_serves_in,p2_first_serve_points_won,p1_first_serve_points_won,p1_second_serves_in,...,p1_name,p2_name,p1_sets_won,p2_sets_won,p1_games_won,p2_games_won,surface,p1_ranking_points,p2_ranking_points,y
0,91.0,7.0,8.0,0.042254,0.061728,0.577465,0.432099,0.800000,0.682927,0.380282,...,Magnus Norman,Thomas Enqvist,1,2,10,15,Hard,2083.0,1489.0,1
1,115.0,10.0,4.0,0.049383,0.012195,0.444444,0.487805,0.650000,0.750000,0.506173,...,Nicolas Escude,Lleyton Hewitt,1,2,11,15,Hard,1026.0,1430.0,1
2,96.0,8.0,7.0,0.012658,0.055556,0.556962,0.416667,0.866667,0.795455,0.430380,...,Thomas Enqvist,Lleyton Hewitt,1,2,11,15,Hard,1489.0,1430.0,1
3,96.0,8.0,5.0,0.022472,0.044118,0.426966,0.514706,0.828571,0.631579,0.550562,...,Sebastien Grosjean,Thomas Enqvist,0,2,11,14,Hard,1187.0,1489.0,1
4,75.0,2.0,2.0,0.090909,0.020000,0.560606,0.620000,0.838710,0.567568,0.348485,...,Tim Henman,Nicolas Escude,0,2,4,12,Hard,1614.0,1026.0,1
